# Connectivity

1. Spectral connectivity
2. Source PSD
3. Envelope correlations

## 1. Spectral connectivity

Adapted from:

https://martinos.org/mne/stable/auto_examples/connectivity/plot_mne_inverse_connectivity_spectrum.html

Compute connectivity between 4 source space labels across the spectrum
between 7.5 Hz and 40 Hz.


In [1]:
import mne
from mne.datasets import sample
from mne.minimum_norm import apply_inverse_epochs, read_inverse_operator
from mne.connectivity import spectral_connectivity

data_path = sample.data_path()
subjects_dir = data_path + '/subjects'
fname_inv = data_path + '/MEG/sample/sample_audvis-meg-oct-6-meg-inv.fif'
fname_raw = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'
fname_event = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw-eve.fif'

%matplotlib qt

In [2]:
# Load data
raw = mne.io.read_raw_fif(fname_raw)
events = mne.read_events(fname_event)
raw.info['bads'] += ['MEG 2443']
event_id = {'Auditory/Left': 1}
epochs = mne.Epochs(raw, events, event_id,
                    reject=dict(mag=4e-12, grad=4000e-13, eog=150e-6),
                    preload=True)

In [3]:
# Compute inverse solution and for each epoch. By using "return_generator=True"
# stcs will be a generator object instead of a list.

inv = read_inverse_operator(fname_inv)
stcs = apply_inverse_epochs(epochs, inv, lambda2=1. / 9.,
                            pick_ori="normal", return_generator=True)
print(stcs)

<generator object _apply_inverse_epochs_gen at 0x116b2cbf8>


In [4]:
# Get the HCP-MMP parcellation labels
mne.datasets.fetch_hcp_mmp_parcellation(verbose=True, subjects_dir=subjects_dir)
# Read them
labels = mne.read_labels_from_annot('fsaverage', 'HCPMMP1', subjects_dir=subjects_dir)
print(len(labels))
print(labels[0])
# Morph them
labels = mne.morph_labels(labels, 'sample', subjects_dir=subjects_dir)
print(len(labels))
print(labels[0])

362
<Label  |  fsaverage, '???-lh', lh : 13868 vertices>
362
<Label  |  sample, '???-lh', lh : 9780 vertices>


In [5]:
# Read four specific labels (A1 and V1 in left and right)
names = ['L_A1_ROI-lh', 'R_A1_ROI-rh', 'L_V1_ROI-lh', 'R_V1_ROI-rh']
label_names = [label.name for label in labels]
labels = [labels[label_names.index(name)] for name in names]
print(labels)

[<Label  |  sample, 'L_A1_ROI-lh', lh : 285 vertices>, <Label  |  sample, 'R_A1_ROI-rh', rh : 260 vertices>, <Label  |  sample, 'L_V1_ROI-lh', lh : 5057 vertices>, <Label  |  sample, 'R_V1_ROI-rh', rh : 5172 vertices>]


In [6]:
# Average the source estimates within each label using sign-flips to reduce
# signal cancellations, also here we return a generator
label_ts = mne.extract_label_time_course(stcs, labels, inv['src'], mode='mean_flip',
                                         return_generator=True)
print(label_ts)

<generator object _gen_extract_label_time_course at 0x834efb780>


In [7]:
# compute the connectivity
fmin, fmax = 7.5, 40.
method = 'wpli2_debiased'  # 'coh', etc.
con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
    label_ts, method=method, sfreq=epochs.info['sfreq'],
    fmin=fmin, fmax=fmax, verbose=True)

Connectivity computation...
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 3)
    Created the whitener using a noise covariance matrix with rank 302 (3 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Picked 305 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 55
Extracting time courses for 4 labels (mode: mean_flip)
only using indices for lower-triangular matrix
    computing connectivity for 6 connections
    using t=0.000s..0.699s for estimation (106 points)
    frequencies: 8.5Hz..39.7Hz (23 points)
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Debiased WPLI Square
    computing connectivity for epoch 1
Processing epoch : 2 / 55
Extracting time courses for 4 labels (mode: mean_flip)
    

In [8]:
# Do some matplotlib to plot the data
import matplotlib.pyplot as plt
n_rows, n_cols = con.shape[:2]
fig, axes = plt.subplots(n_rows, n_cols, sharex=True, sharey=True)
for i in range(n_rows):
    for j in range(i + 1):
        if i == j:
            fig.delaxes(axes[i, j])
            continue
        axes[i, j].plot(freqs, con[i, j, :])
        axes[j, i].plot(freqs, con[i, j, :])

        if j == 0:
            axes[i, j].set_ylabel(names[i])
            axes[0, i].set_title(names[i])
        if i == (n_rows - 1):
            axes[i, j].set_xlabel(names[j])
        axes[i, j].set(xlim=[fmin, fmax], ylim=[-0.1, 1])
        axes[j, i].set(xlim=[fmin, fmax], ylim=[-0.1, 1])
        for f in [8, 12, 18, 35]:
            axes[i, j].axvline(f, color='k')
            axes[j, i].axvline(f, color='k')
plt.tight_layout()
plt.show()

## 2. Source PSD analysis

- Compute the resting state PSD from raw data.
- Mostly follow the Brainstorm OMEGA resting tutorial pipeline:
  1. Filtering: downsample heavily.
  2. Artifact detection: use SSP for EOG and ECG.
  3. Source localization: dSPM, depth weighting, cortically constrained.
  4. Frequency: power spectrum density (Welch), 4 sec window, 50% overlap.
  5. Standardize: normalize by relative power for each source.

In [9]:
import os.path as op
data_path = mne.datasets.opm.data_path(verbose=True)
subject = 'OPM_sample'
subjects_dir = op.join(data_path, 'subjects')
bem_dir = op.join(subjects_dir, subject, 'bem')
bem_fname = op.join(subjects_dir, subject, 'bem',
                    subject + '-5120-5120-5120-bem-sol.fif')
src_fname = op.join(bem_dir, '%s-oct6-src.fif' % subject)
raw_fname = data_path + '/MEG/SQUID/SQUID_resting_state.fif'
erm_fname = data_path + '/MEG/SQUID/SQUID_empty_room.fif'
trans_fname = data_path + '/MEG/SQUID/SQUID-trans.fif'

In [10]:
new_sfreq = 90.  # Nyquist frequency (45 Hz) < line noise freq (50 Hz)
raw = mne.io.read_raw_fif(raw_fname, verbose='error')  # ignore naming
print(raw)
raw.load_data().resample(new_sfreq)
print(raw)
raw.info['bads'] = ['MEG2233', 'MEG1842']
raw_erm = mne.io.read_raw_fif(erm_fname, verbose='error')
raw_erm.load_data().resample(new_sfreq)
raw_erm.info['bads'] = ['MEG2233', 'MEG1842']
print(raw_erm)

<Raw  |  SQUID_resting_state.fif, n_channels x n_times : 310 x 62000 (62.0 sec), ~6.8 MB, data not loaded>
<Raw  |  SQUID_resting_state.fif, n_channels x n_times : 310 x 5580 (62.0 sec), ~20.0 MB, data loaded>
<Raw  |  SQUID_empty_room.fif, n_channels x n_times : 310 x 5490 (61.0 sec), ~13.9 MB, data loaded>


In [11]:
ssp_ecg, _ = mne.preprocessing.compute_proj_ecg(
    raw, tmin=-0.1, tmax=0.1, n_grad=1, n_mag=1)
raw.add_proj(ssp_ecg, remove_existing=True)
ssp_ecg_eog, _ = mne.preprocessing.compute_proj_eog(
    raw, n_grad=1, n_mag=1, ch_name='MEG0112')
raw.add_proj(ssp_ecg_eog, remove_existing=True)
raw_erm.add_proj(ssp_ecg_eog)
fig = mne.viz.plot_projs_topomap(raw.info['projs'][-4:],
                                 info=raw.info)
fig.subplots_adjust(0.05, 0.05, 0.95, 0.85)

In [12]:
from mne.filter import next_fast_len
n_fft = next_fast_len(int(round(4 * new_sfreq)))
print('Using n_fft=%d (%0.1f sec)' % (n_fft, n_fft / raw.info['sfreq']))
fig = raw.plot_psd(n_fft=n_fft, proj=True)
fig.subplots_adjust(0.1, 0.1, 0.95, 0.85)

Using n_fft=360 (4.0 sec)


In [13]:
src = mne.read_source_spaces(src_fname)
bem = mne.read_bem_solution(bem_fname)
fig = mne.viz.plot_alignment(
    raw.info, trans=trans_fname, subject=subject,
    subjects_dir=subjects_dir, dig=True, coord_frame='meg',
    surfaces=('head', 'white'))
from mayavi import mlab
mlab.view(0, 90, focalpoint=(0., 0., 0.), distance=0.6, figure=fig)
fig

In [14]:
fwd = mne.make_forward_solution(
    raw.info, trans_fname, src, bem, eeg=False, verbose=True)

Source space          : <SourceSpaces: [<surface (lh), n_vertices=169022, n_used=4098, coordinate_frame=MRI (surface RAS)>, <surface (rh), n_vertices=169992, n_used=4098, coordinate_frame=MRI (surface RAS)>]>
MRI -> head transform : /Users/larsoner/mne_data/MNE-OPM-data/MEG/SQUID/SQUID-trans.fif
Measurement data      : instance of Info
Conductor model   : instance of ConductorModel
Accurate field computations
Do computations in head coordinates
Free source orientations

Read 2 source spaces a total of 8196 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.995623 -0.029776 -0.088592       1.15 mm
     0.062622  0.916188  0.395825       5.31 mm
     0.069381 -0.399641  0.914042      25.88 mm
     0.000000  0.000000  0.000000       1.00

Read 306 MEG channels from info
84 coil definitions read
Coordinate transformation: MEG device -> head
     0.993107 -0.074371 -0.090590       0.76 mm
     0.079171  0.995577  0.050589     -13.97 mm
     0.086427 -0.0574

In source space:

1. Compute and apply inverse to PSD estimated using multitaper + Welch.
2. Group into frequency bands
3. Normalize each source point independently (by total power across freq bands)

This makes the value of each sensor point and source location in each frequency band the percentage of the PSD accounted for by that band.

In [15]:
snr = 3.
lambda2 = 1. / snr ** 2
noise_cov = mne.compute_raw_covariance(raw_erm)
inv = mne.minimum_norm.make_inverse_operator(
    raw.info, forward=fwd, noise_cov=noise_cov, verbose=True)
print(inv)

Converting forward solution to surface orientation
    Average patch normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
info["bads"] and noise_cov["bads"] do not match, excluding bad channels from both
Computing inverse operator with 304 channels.
    304 out of 306 channels remain after picking
Selected 304 channels
Creating the depth weighting matrix...
    202 planar channels
    limit = 7839/8196 = 10.032537
    scale = 3.34919e-08 exp = 0.8
Applying loose dipole orientations. Loose value of 0.2.
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 17)
Computing data rank from covariance with rank=None
    Using tolerance 7.1e-13 (2.2e-16 eps * 304 dim * 10  max singular value)
    Estimated rank (mag + grad): 287
    MEG: rank 287 computed from 304 data channels with 17 projectors
    Setting small MEG eigenvalues to zero (without PCA)
Creating the source covar

In [16]:
stc_psd = mne.minimum_norm.compute_source_psd(
    raw, inv, lambda2=lambda2, n_fft=n_fft, verbose=True)

31 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
Created an SSP operator (subspace dimension = 17)
17 projection items activated
Considering frequencies 0 ... 200 Hz
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 17)
    Created the whitener using a noise covariance matrix with rank 287 (17 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Picked 304 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Reducing data rank 304 -> 287
Using hann windowing on at most 31 epochs
[............................................................] 100.00%  |   


In [17]:
stc_norm = stc_psd.sum()  # sum across freqs
print(stc_norm.data.shape)
freq_bands = dict(alpha=(8, 12), beta=(15, 29))
stcs = dict()
# Normalize each source point by the total power across freqs
for band, limits in freq_bands.items():
    stcs[band] = 100 * stc_psd.copy().crop(*limits).sum() / stc_norm.data

(8196, 1)


In [18]:
def plot_band(band):
    title = "%s\n(%d-%d Hz)" % ((band,) + freq_bands[band])
    brain = stcs[band].plot(
        subject=subject, subjects_dir=subjects_dir, views='cau', hemi='both',
        time_label=title, title=title, colormap='inferno',
        clim=dict(kind='percent', lims=(70, 85, 99)))
    brain.show_view(dict(azimuth=0, elevation=0), roll=0)
    return brain


brain_theta = plot_band('alpha')

In [19]:
brain_beta = plot_band('beta')

## Envelope correlation

Compute envelope correlations of orthogonalized activity in source space using resting state CTF data.

1. Hipp JF, Hawellek DJ, Corbetta M, Siegel M, Engel AK (2012)
   Large-scale cortical correlation structure of spontaneous
   oscillatory activity. Nature Neuroscience 15:884–890

2. Khan S et al. (2018). Maturation trajectories of cortical
   resting-state networks depend on the mediating frequency band.
   Neuroimage 174:57–68

In [20]:
import os.path as op
import mne
data_path = mne.datasets.brainstorm.bst_resting.data_path(verbose=True)
subjects_dir = op.join(data_path, 'subjects')
subject = 'bst_resting'
trans = op.join(data_path, 'MEG', 'bst_resting', 'bst_resting-trans.fif')
src = op.join(subjects_dir, subject, 'bem', subject + '-oct-6-src.fif')
bem = op.join(subjects_dir, subject, 'bem', subject + '-5120-bem-sol.fif')
raw_fname = op.join(data_path, 'MEG', 'bst_resting',
                    'subj002_spontaneous_20111102_01_AUX.ds')

In [21]:
# Load data, crop, and downsample for speed
raw = mne.io.read_raw_ctf(raw_fname, verbose='error').crop(0, 120).load_data()
print(raw)
raw.pick_types(meg=True, eeg=False).resample(80)
print(raw)

<RawCTF  |  subj002_spontaneous_20111102_01_AUX.meg4, n_channels x n_times : 301 x 288001 (120.0 sec), ~662.3 MB, data loaded>
<RawCTF  |  subj002_spontaneous_20111102_01_AUX.meg4, n_channels x n_times : 298 x 9600 (120.0 sec), ~22.7 MB, data loaded>


In [22]:
# Do some preprocessing
from mne.preprocessing import compute_proj_ecg, compute_proj_eog
raw.apply_gradient_compensation(3)
projs_ecg, _ = compute_proj_ecg(raw, n_grad=1, n_mag=2)
projs_eog, _ = compute_proj_eog(raw, n_grad=1, n_mag=2, ch_name='MLT31-4407')
raw.add_proj(projs_ecg)
raw.add_proj(projs_eog)
raw.plot()
plt.show()

In [23]:
# compute covariance before band-pass of interest,
# or (probably better) use empty-room covariance
cov = mne.compute_raw_covariance(raw)
print(cov)

<Covariance  |  size : 272 x 272, n_samples : 9584, data : [[ 3.09347942e-26  3.05272666e-26  2.84488098e-26 ... -9.02647780e-27
  -7.86630092e-27 -6.95465009e-27]
 [ 3.05272666e-26  3.76904465e-26  3.94508048e-26 ... -1.50856682e-26
  -1.25770898e-26 -7.33855887e-27]
 [ 2.84488098e-26  3.94508048e-26  4.57821276e-26 ... -1.58431145e-26
  -1.36825027e-26 -6.65663964e-27]
 ...
 [-9.02647780e-27 -1.50856682e-26 -1.58431145e-26 ...  5.69583276e-26
   4.02453939e-26  1.54137153e-26]
 [-7.86630092e-27 -1.25770898e-26 -1.36825027e-26 ...  4.02453939e-26
   3.62590596e-26  9.20486900e-27]
 [-6.95465009e-27 -7.33855887e-27 -6.65663964e-27 ...  1.54137153e-26
   9.20486900e-27  2.42454539e-26]]>


In [24]:
raw.filter(14, 30)
events = mne.make_fixed_length_events(raw, duration=10.)
epochs = mne.Epochs(raw, events=events, tmin=0, tmax=10.,
                    baseline=None, reject=dict(mag=8e-13),
                    preload=True)
print(epochs)

<Epochs  |   10 events (all good), 0 - 10 sec, baseline off, ~19.2 MB, data loaded,
 '1': 10>


In [25]:
# Create forward
fwd = mne.make_forward_solution(epochs.info, trans, src, bem, verbose=True)

Source space          : /Users/larsoner/mne_data/MNE-brainstorm-data/bst_resting/subjects/bst_resting/bem/bst_resting-oct-6-src.fif
MRI -> head transform : /Users/larsoner/mne_data/MNE-brainstorm-data/bst_resting/MEG/bst_resting/bst_resting-trans.fif
Measurement data      : instance of Info
Conductor model   : /Users/larsoner/mne_data/MNE-brainstorm-data/bst_resting/subjects/bst_resting/bem/bst_resting-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading /Users/larsoner/mne_data/MNE-brainstorm-data/bst_resting/subjects/bst_resting/bem/bst_resting-oct-6-src.fif...
Read 2 source spaces a total of 8196 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999797 -0.005775 -0.019288       2.71 mm
     0.011390  0.952195  0.305279      16.66 mm
     0.016602 -0.305437  0.952068      28.47 mm
     0.000000  0.000000  0.000000       1.00

Read 272 MEG channels from info
Read  26 MEG compensation chann

In [26]:
# Create inverse
inv = mne.minimum_norm.make_inverse_operator(epochs.info, fwd, cov, verbose=True)

Converting forward solution to surface orientation
    Average patch normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 272 channels.
    272 out of 272 channels remain after picking
Removing 5 compensators from info because not all compensation channels were picked.
Selected 272 channels
Creating the depth weighting matrix...
    272 magnetometer or axial gradiometer channels
    limit = 8074/8196 = 10.006553
    scale = 5.92189e-11 exp = 0.8
Applying loose dipole orientations. Loose value of 0.2.
Whitening the forward solution.
Removing 5 compensators from info because not all compensation channels were picked.
    Created an SSP operator (subspace dimension = 4)
Computing data rank from covariance with rank=None
    Using tolerance 7.5e-14 (2.2e-16 eps * 272 dim * 1.2  max singular value)
    Estimated rank (mag): 268
    MAG: rank 268 computed from 272 data c

In [27]:
# Take the Hilbert transform of epochs
epochs.apply_hilbert()

# Compute label (complex-valued) time series
labels = mne.read_labels_from_annot(subject, 'aparc_sub',
                                    subjects_dir=subjects_dir)
stcs = mne.minimum_norm.apply_inverse_epochs(
    epochs, inv, lambda2=1. / 9., pick_ori='normal',
    return_generator=True)
label_ts = mne.extract_label_time_course(
    stcs, labels, inv['src'], return_generator=True)
print(label_ts)

<generator object _gen_extract_label_time_course at 0x1c6b96bdb0>


In [28]:
# Compute envelope correlations!
corr = mne.connectivity.envelope_correlation(label_ts, verbose=True)

Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 4)
    Created the whitener using a noise covariance matrix with rank 268 (4 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Picked 272 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 10
Extracting time courses for 450 labels (mode: mean_flip)
Processing epoch : 2 / 10
Extracting time courses for 450 labels (mode: mean_flip)
Processing epoch : 3 / 10
Extracting time courses for 450 labels (mode: mean_flip)
Processing epoch : 4 / 10
Extracting time courses for 450 labels (mode: mean_flip)
Processing epoch : 5 / 10
Extracting time courses for 450 labels (mode: mean_flip)
Processing epoch : 6 / 10
Extracting time courses for 450 labels (mode: mean_flip)
Processing epoch : 7 / 10
Extracting time courses f

In [ ]:
# Plot the 2D correlation matrix
print(corr.shape)
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(figsize=(4, 4))
ax.imshow(corr, cmap='viridis', clim=np.percentile(corr, [5, 95]))
fig.tight_layout()

In [29]:
# Compute the degree
threshold_prop = 0.15  # percentage of strongest edges to keep in the graph
degree = mne.connectivity.degree(corr, threshold_prop=threshold_prop)
print(degree.shape)

(450,)


In [30]:
# Turn our label data to a STC
stc = mne.labels_to_stc(labels, degree)
print(stc)
# Restrict it back to our original source space points
stc = stc.in_label(mne.Label(inv['src'][0]['vertno'], hemi='lh') +
                   mne.Label(inv['src'][1]['vertno'], hemi='rh'))
print(stc)

<SourceEstimate  |  312523 vertices, subject : bst_resting, tmin : 0.0 (ms), tmax : 0.0 (ms), tstep : 1000.0 (ms), data shape : (312523, 1)>
<SourceEstimate  |  8196 vertices, subject : bst_resting, tmin : 0.0 (ms), tmax : 0.0 (ms), tstep : 1000.0 (ms), data shape : (8196, 1)>


In [31]:
# Plot the result
brain = stc.plot(
    clim=dict(kind='percent', lims=[75, 85, 95]), colormap='gnuplot',
    subjects_dir=subjects_dir, views='dorsal', hemi='both',
    smoothing_steps=25, time_label='Beta band')
brain

<Brain subject_id="bst_resting", hemi="both", surf="inflated">
